In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import os
import sys
from ipywidgets import widgets
from IPython.display import display, HTML
from IPython.display import clear_output
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
from tabulate import tabulate

# Customized modules
HOME = '/home/serinatan/project/GPU-Virtualization-Benchmarks/util'
if HOME not in sys.path:
    sys.path.append(HOME)

import data.scripts.common.format as fmt
import data.scripts.common.constants as const
import data.scripts.gen_tables.gen_pair_configs as gen_pair
import data.scripts.gen_tables.search_best_inter as search_inter
import data.scripts.gen_graphs.gen_altair_timeline as gen_altair

SCRIPT_PATH = os.path.join(HOME, 'data/scripts') 
PKL_PATH = os.path.join(HOME, 'data/pickles') 
CSV_PATH = os.path.join(HOME, 'data/csv') 
GRAPH_PATH = os.path.join(HOME, 'data/graphs')

%config InlineBackend.figure_format ='retina'
plt.style.use('seaborn-talk')

In [2]:
def draw_table(df, cols, hide_index=True):
    if hide_index:
        return df[cols].style.set_table_styles(fmt.table_style).hide_index()
    else:
        return df[cols].style.set_table_styles(fmt.table_style)

# Seq

In [3]:
# run python scripts to generate all the pickles needed
gen_seq = os.path.join(SCRIPT_PATH, 'gen_tables/gen_table_seq.py')   
seq_file = os.path.join(CSV_PATH, 'seq-multi.csv')
seq_pkl = os.path.join(PKL_PATH, 'seq-multi.pkl')
%run $gen_seq --multi --csv $seq_file --output $seq_pkl

df_seq = pd.read_pickle(os.path.join(PKL_PATH, 'seq-multi.pkl'))
print(df_seq.columns)

Index(['pair_str', 'config', 'gpusim_version', 'jobId', 'stall_icnt_to_l2',
       'stall_l2_to_icnt', 'stall_core_ldst', 'l1D_miss_rate', 'l2_miss_rate',
       'l2_rshr_entry_fail', 'l2_rshr_merge_fail', 'l2_total_accesses',
       'mem_count', 'empty_warp', 'stall_warp', 'idle_warp', 'scoreboard_warp',
       'tot_warp_insn', 'runtime', 'instructions', 'l2_bw', 'avg_mem_lat',
       'avg_core_to_l2', 'avg_l2_to_core', 'avg_mrq_latency', 'dram_eff',
       'dram_bw', 'row_buffer_locality', 'mem_idle', 'total_cmd', 'ipc',
       'avg_dram_bw', 'avg_dram_eff', 'avg_row_locality', 'std_dram_bw',
       'ratio_dram_bw', 'MPKI', 'l2_access_density', '1_kidx', 'waves'],
      dtype='object')


In [27]:
col_seq = ['pair_str', '1_kidx', 'runtime',
           'ipc',
           'avg_dram_bw', 
           'l1D_miss_rate',
           'l2_miss_rate',
          ]

draw_table(df_seq, col_seq).format({
    'runtime': '{:,}',
    'avg_dram_bw':'{:.4f}', 
    'sp_busy': '{:.2f}',
    'dp_busy': '{:.2f}',
    'int_busy': '{:.2f}',
    'tensor_busy': '{:.2f}',
    'sfu_busy': '{:.2f}',
})

pair_str,1_kidx,runtime,ipc,avg_dram_bw,l1D_miss_rate,l2_miss_rate
nvd_conv-0,1,"190,346",2131.9,0.7242,0.9991,0.931
nvd_conv-0,2,"208,164",1755.92,0.6698,0.9997,0.9323
parb_sad-0,1,"51,633",2686.45,0.0061,1,0.6109
parb_sad-0,2,"21,556",203.199,0.4558,0.9099,0.9849
parb_sad-0,3,"12,806",89.9086,0.1249,0.9109,0.9371
parb_sad-1,1,"3,000,706",3754.08,0.1736,0.9999,0.5524
parb_sad-1,2,"2,324,970",153.001,0.5645,0.9121,0.9998
parb_sad-1,3,"541,865",172.562,0.5842,0.912,0.9992


# Intra

In [5]:
gen_intra = os.path.join(SCRIPT_PATH, 'gen_tables/gen_table_intra.py')
intra_file = os.path.join(CSV_PATH, 'intra-multi.csv')
intra_pkl = os.path.join(PKL_PATH, 'intra-multi.pkl')

%run $gen_intra --out_intra $intra_pkl --seq $seq_pkl --csv $intra_file

df_intra = pd.read_pickle(intra_pkl)
df_intra.sort_values(['pair_str', '1_kidx'], inplace=True)
# col_intra = ['pair_str', '1_kidx', 'regs', 'thread_count', 'smem', 'intra']
# draw_table(df_intra, col_intra)

In [6]:
col_prod = ['pair_str_x', '1_kidx_x', 'pair_str_y', '1_kidx_y',
            'norm_ipc_x', 'norm_ipc_y', 'diff_mflat', 'sum_ipc', 
            'intra_x', 'intra_y', 
           'sum_comp', 'sum_dram',]
find_pair = os.path.join(SCRIPT_PATH, 'gen_tables/gen_pair_configs.py')
pairs = df_seq.apply(lambda row: ':'.join([row['pair_str'], str(row['1_kidx'])]), axis=1)

In [7]:
app1 = widgets.Dropdown(options=pairs, value=pairs[0], description='App 1:')
app2 = widgets.Dropdown(options=pairs, value=pairs[1], description='App 2:')

qos = widgets.FloatSlider(
    value=0.75,
    min=0.1,
    max=0.95,
    step=0.05,
    description='QoS:',
    readout_format='.2f',
)

button = widgets.Button(description='Calculate', button_style='info')

def show_widgets():
    display(app1)
    display(app2)
    display(qos)
    display(button)
    

def onclick(b):
    clear_output()
    show_widgets()
    %run $find_pair --app $app1.value $app2.value --qos $qos.value --intra_pkl $intra_pkl --top
    
    df_prod = pd.read_pickle(os.path.join(PKL_PATH, 'pair_candidates.pkl'))
   
    display(HTML(draw_table(df_prod, col_prod, False).render()))
    
    

In [8]:
show_widgets()
button.on_click(onclick)

Dropdown(description='App 1:', index=6, options=('nvd_conv-0:1', 'nvd_conv-0:2', 'parb_sad-0:1', 'parb_sad-0:2…

Dropdown(description='App 2:', index=7, options=('nvd_conv-0:1', 'nvd_conv-0:2', 'parb_sad-0:1', 'parb_sad-0:2…

FloatSlider(value=0.75, description='QoS:', max=0.95, min=0.1, step=0.05)

Button(button_style='info', description='Calculate', style=ButtonStyle())

# CTX Ratio

In [9]:
gen_pair = os.path.join(SCRIPT_PATH, 'gen_tables/gen_table_pair.py')
ctx_csv = os.path.join(HOME, 'data/csv/ctx.csv')
output = os.path.join(PKL_PATH, 'pair_ctx.pkl')


In [10]:
%run $gen_pair --csv $ctx_csv --output $output --seq_pkl $seq_pkl --qos 0.5 --multi --how ctx

Index(['pair_str', 'config', 'gpusim_version', 'jobId', 'stall_icnt_to_l2',
       'stall_l2_to_icnt', 'stall_core_ldst', 'l1D_miss_rate', 'l2_miss_rate',
       'l2_rshr_entry_fail', 'l2_rshr_merge_fail', 'l2_total_accesses',
       'mem_count', 'empty_warp', 'stall_warp', 'idle_warp', 'scoreboard_warp',
       'tot_warp_insn', 'runtime', 'instructions', 'l2_bw', 'avg_mem_lat',
       'avg_core_to_l2', 'avg_l2_to_core', 'avg_mrq_latency', 'dram_eff',
       'dram_bw', 'row_buffer_locality', 'mem_idle', 'total_cmd'],
      dtype='object')


In [11]:
df_ctx= pd.read_pickle(output)
uniq_pairs = df_ctx[['1_bench', '2_bench']].drop_duplicates().values
uniq_pairs = ['+'.join(x) for x in uniq_pairs]

col_ctx = ['1_bench', '2_bench', '1_ctx', '2_ctx', 
           'cta_quota',
#            'runtime', 'norm_runtime'
          ]
draw_table(df_ctx, col_ctx)

1_bench,2_bench,1_ctx,2_ctx,cta_quota
parb_sad-0,nvd_conv-0,0.5,0.5,"[[], [8, 2, 2], [16, 8]]"
parb_sad-0,nvd_conv-0,0.25,0.75,"[[], [4, 2, 2], [24, 12]]"
parb_sad-0,nvd_conv-0,0.75,0.25,"[[], [12, 2, 2], [8, 4]]"
parb_sad-1,nvd_conv-0,0.5,0.5,"[[], [8, 8, 16], [16, 8]]"
parb_sad-1,nvd_conv-0,0.25,0.75,"[[], [4, 4, 8], [24, 12]]"
parb_sad-1,nvd_conv-0,0.75,0.25,"[[], [12, 12, 24], [8, 4]]"


In [22]:
ctx_dd = widgets.Dropdown(options=uniq_pairs, value=uniq_pairs[0], description='Pair:')

ctx_btn = widgets.Button(description='Display', button_style='info')

def show_widgets():
    display(ctx_dd)
    display(ctx_btn)
    

def ctx_onclick(b):
    clear_output()
    show_widgets()
    
    apps = ctx_dd.value.split('+')
    df_apps = df_ctx[(df_ctx['1_bench'] == apps[0]) & (df_ctx['2_bench'] == apps[1])]
    df_apps = df_apps.sort_values('1_ctx')
    for rid, row in df_apps.iterrows():
        gen_altair.draw_altair_timeline(row, '1_ctx').display()  
        # Print ctas/SM for each kernel in each app
        print('Note:')
        print(row['1_bench'], ':', row['cta_quota'][1], 'CTAs/SM')
        print(row['2_bench'], ':', row['cta_quota'][2], 'CTAs/SM')
        
#         print(row['norm_ipc'])
#         print(row['baseline'])
        print(row['runtime'])
        print('slowdown:', row['sld'])
        print('weighted speedup:', row['ws'])
        print('-' * 100)

show_widgets()
ctx_btn.on_click(ctx_onclick)
 

Dropdown(description='Pair:', options=('parb_sad-0+nvd_conv-0', 'parb_sad-1+nvd_conv-0'), value='parb_sad-0+nv…

Button(button_style='info', description='Display', style=ButtonStyle())

alt.LayerChart(...)

Note:
parb_sad-0 : [4, 2, 2] CTAs/SM
nvd_conv-0 : [24, 12] CTAs/SM
[[], [120682, 65436, 37453, 88098, 53440, 44124, 74405, 24826, 9068], [224169, 250737]]
slowdown: [0, 0.4202740248220451, 0.8391344813499935]
weighted speedup: 1.2594085061720386
----------------------------------------------------------------------------------------------------


alt.LayerChart(...)

Note:
parb_sad-0 : [8, 2, 2] CTAs/SM
nvd_conv-0 : [16, 8] CTAs/SM
[[], [79904, 60179, 39817, 54554, 51112, 36158, 51233, 49403, 35035, 44531, 24691, 9164], [222775, 257227]]
slowdown: [0, 0.564031089102417, 0.8302257073928858]
weighted speedup: 1.3942567964953028
----------------------------------------------------------------------------------------------------


alt.LayerChart(...)

Note:
parb_sad-0 : [12, 2, 2] CTAs/SM
nvd_conv-0 : [8, 4] CTAs/SM
[[], [57535, 58037, 41430, 48948, 41217, 17930, 41243, 39681, 22359, 41004, 39197, 22264, 41731, 28251, 9066], [233726, 296690]]
slowdown: [0, 0.730558888806295, 0.7513159482368555]
weighted speedup: 1.4818748370431507
----------------------------------------------------------------------------------------------------


# Pair-Dynamic

In [13]:
dynamic_csv = os.path.join(CSV_PATH, 'pair_dynamic_multi.csv')
dynamic_output = os.path.join(PKL_PATH, 'pair_dynamic_multi.pkl')
%run $gen_pair --csv $dynamic_csv --output $dynamic_output --seq_pkl $seq_pkl --qos 0.5 --how dynamic --multi --isolated_pkl $intra_pkl

Index(['pair_str', 'config', 'gpusim_version', 'jobId', 'stall_icnt_to_l2',
       'stall_l2_to_icnt', 'stall_core_ldst', 'l1D_miss_rate', 'l2_miss_rate',
       'l2_rshr_entry_fail', 'l2_rshr_merge_fail', 'l2_total_accesses',
       'mem_count', 'empty_warp', 'stall_warp', 'idle_warp', 'scoreboard_warp',
       'tot_warp_insn', 'runtime', 'instructions', 'l2_bw', 'avg_mem_lat',
       'avg_core_to_l2', 'avg_l2_to_core', 'avg_mrq_latency', 'dram_eff',
       'dram_bw', 'row_buffer_locality', 'mem_idle', 'total_cmd'],
      dtype='object')


In [14]:
df_dynamic = pd.read_pickle(dynamic_output)
df_best = df_dynamic.sort_values('ws').drop_duplicates(['1_bench', '1_kidx', '2_bench', '2_kidx'])

col_best = ['1_bench', '1_kidx', '1_intra', '2_bench', '2_kidx', '2_intra', 'sld', 'ws']
draw_table(df_best, col_best)

1_bench,1_kidx,1_intra,2_bench,2_kidx,2_intra,sld,ws
parb_sad-1,3,12,nvd_conv-0,1,16,"[0, 0.6081810631229236, 0.4447689769748064]",1.05295
parb_sad-1,2,4,nvd_conv-0,2,8,"[0, 0.6549559962488144, 0.44567193130070076]",1.10063
parb_sad-1,2,4,nvd_conv-0,1,12,"[0, 0.6733531740974924, 0.42887431364022466]",1.10223
parb_sad-1,3,8,nvd_conv-0,2,10,"[0, 0.6267842353409306, 0.49395383271954135]",1.12074
parb_sad-0,2,2,nvd_conv-0,2,10,"[0, 0.51141162514828, 0.6167401925202134]",1.12815
parb_sad-0,2,2,nvd_conv-0,1,20,"[0, 0.5388057090009248, 0.5973494512805546]",1.13616
parb_sad-0,1,4,nvd_conv-0,2,2,"[0, 0.7312938177182919, 0.5808082499079251]",1.3121
parb_sad-0,3,2,nvd_conv-0,1,20,"[0, 0.5678683872112101, 0.8138757290187962]",1.38174
parb_sad-1,1,8,nvd_conv-0,1,12,"[0, 0.5338318149391486, 0.866422081823644]",1.40025
parb_sad-0,3,1,nvd_conv-0,2,4,"[0, 0.4633140376266281, 0.9376081003170943]",1.40092


In [15]:
df_best.set_index(['1_bench', '1_kidx', '2_bench', '2_kidx'], inplace=True)

In [25]:

best_dd = widgets.Dropdown(options=uniq_pairs, value=uniq_pairs[0], description='Pair:')
best_btn = widgets.Button(description='Best', button_style='info')

def show_widgets():
    display(best_dd)
    display(best_btn)
    

def best_onclick(b):
    clear_output()
    show_widgets()
    
    apps = best_dd.value.split('+')
    def get_best_config(app1, app2):
        configs_1 = np.zeros((len(const.kernel_yaml[app1]), len(const.kernel_yaml[app2]))).tolist()
        configs_2 = np.zeros((len(const.kernel_yaml[app1]), len(const.kernel_yaml[app2]))).tolist()
        for kernel_1 in const.kernel_yaml[app1]:
            for kernel_2 in const.kernel_yaml[app2]:
                cta_1 = df_best.loc[(app1, kernel_1, app2, kernel_2), '1_intra']
                sld_1 = df_best.loc[(app1, kernel_1, app2, kernel_2), 'sld'][1]
                cta_2 = df_best.loc[(app1, kernel_1, app2, kernel_2), '2_intra']
                sld_2 = df_best.loc[(app1, kernel_1, app2, kernel_2), 'sld'][2]
                configs_1[kernel_1-1][kernel_2-1] = '{:d}, {:.2f}'.format(int(cta_1), sld_1)
                configs_2[kernel_1-1][kernel_2-1] = '{:d}, {:.2f}'.format(int(cta_2), sld_2)
#         return [configs_1.astype(np.int64), configs_2.astype(np.int64)]
        return [configs_1, configs_2]
                
    configs = get_best_config(apps[0], apps[1])
    app1_kernels = ['{0}:{1}'.format(apps[0], kernel) for kernel in const.kernel_yaml[apps[0]]]
    app2_kernels = ['{0}:{1}'.format(apps[1], kernel) for kernel in const.kernel_yaml[apps[1]]]
    print(apps[0], 'configs:')
    print(pd.DataFrame(configs[0], app1_kernels, app2_kernels))
    
    print('-' * 50)
    print(apps[1], 'configs:')
    print(pd.DataFrame(configs[1], app1_kernels, app2_kernels))
    

show_widgets()
best_btn.on_click(best_onclick)
 

Dropdown(description='Pair:', options=('parb_sad-0+nvd_conv-0', 'parb_sad-1+nvd_conv-0'), value='parb_sad-0+nv…

Button(button_style='info', description='Best', style=ButtonStyle())

parb_sad-0 configs:
             nvd_conv-0:1 nvd_conv-0:2
parb_sad-0:1      4, 0.66      4, 0.73
parb_sad-0:2      2, 0.54      2, 0.51
parb_sad-0:3      2, 0.57      1, 0.46
--------------------------------------------------
nvd_conv-0 configs:
             nvd_conv-0:1 nvd_conv-0:2
parb_sad-0:1      4, 0.88      2, 0.58
parb_sad-0:2     20, 0.60     10, 0.62
parb_sad-0:3     20, 0.81      4, 0.94


In [17]:
arr = np.array([[], [1, 3], [3, 2, 4]])
for idx, stream in enumerate(arr):
    print(idx, stream)
sum_runtime = np.array([sum(arr_run) for arr_run in arr])

np.min(sum_runtime[sum_runtime > 0])

0 []
1 [1, 3]
2 [3, 2, 4]


4